In [138]:
using Plots
gr()

Plots.GRBackend()

In [182]:
using MLDataPattern
using LIBSVM
using AxisArrays
using JuMP

In [173]:
include("main.jl")

c

In [189]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)
time = Axis{:time}(linspace(0, 9 * double_integrator.Δt, 10))
side = Axis{:side}([:left, :right])
model = c.MPC.create_model(double_integrator, time, side);
numresults = 10000
results = @time(c.collect_data(model, numresults));

527.714614 seconds (15.17 M allocations: 7.237 GB, 0.19% gc time)


In [190]:
examples = convert(Vector{c.Example}, results);

labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);

In [191]:
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [192]:
@time svm = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(svm, data_test)

 19.582709 seconds (4.58 k allocations: 13.378 MB, 1.21% gc time)
  6.843225 seconds (4.52 k allocations: 22.976 MB, 2.08% gc time)


([10,1,15,1,1,10,12,2,16,10  …  4,2,1,1,13,9,6,4,2,9],
[0.000273839 0.00674988 … 0.0189787 0.0622613; 0.316061 0.718599 … 0.026082 0.0622214; … ; 0.0200733 0.108221 … 0.00160833 0.00600236; 0.000114398 0.0234509 … 0.00232755 0.00721533])

In [193]:
mean(predicted_labels .== labels_test)

0.610877824435113

In [194]:
q0 = 2 * rand() - 1
v0 = 4 * rand() - 2
qlimb0 = 2 * rand() - 1
state = c.MPC.State(q0, v0, qlimb0)
contact = AxisArray(rand(Bool, 2, length(time)), side, time)

2-dimensional AxisArray{Bool,2,...} with axes:
    :side, Symbol[:left,:right]
    :time, linspace(0.0,0.9,10)
And data, a 2×10 Array{Bool,2}:
 false  false  true  true  false   true  false   true  false  true
  true  false  true  true   true  false   true  false  false  true

In [195]:
function svmsolve(svm::LIBSVM.SVM, model::c.MPC.MPCModel, state::c.MPC.State, contact)
    visited = Set([BitArray(contact)])
    for i in 1:100
        result = c.MPC.solve!(model, state, contact_sequence=contact)
        @show result.status
        @show getvalue(getobjective(result.model.m))
        if result.status != :Optimal
            result = c.MPC.solve!(model, state, contact_sequence=contact, relax=true)
            @show getvalue(getobjective(result.model.m))
        end

        features = vcat(vec(contact), result.model.m.linconstrDuals)
        _, decision_values = svmpredict(svm, reshape(features, length(features), 1))
        actions = svm.labels[reverse(sortperm(vec(decision_values)))]
        for action in actions
            @show action
            contact[action] = !contact[action]
            b = BitArray(contact)
            if !(b in visited)
                push!(visited, b)
                break
            else
                contact[action] = !contact[action]  # put it back
            end
        end
    end
end

svmsolve (generic function with 1 method)

In [196]:
svmsolve(svm, model, state, contact)

result.status = :Optimal
getvalue(getobjective(result.model.m)) = 88.01498861102404
action = 2
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 95.31931372251242
action = 2
action = 1
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 95.49484952559881
action = 2
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 88.9456435408566
action = 2
action = 1
action = 3
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 89.43196463404406
action = 2
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 95.71839276123181
action = 2
action = 1
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 95.62938313211635
action = 2
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 88.71537293113494
action = 2
action = 1
action = 3
action = 4
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 84.85281790377039
action = 2
result.status = :Optimal
getvalue(getobjective(result.model.m)) = 91.

In [199]:
function topn(n)
    [svm.labels[reverse(sortperm(decision_values[:, i]))[1:n]] for i in 1:size(decision_values, 2)]
end

topn (generic function with 1 method)

In [200]:
mean(getindex.(topn(1), 1) .== labels_test)

0.610877824435113

In [201]:
t = topn(5)
mean([labels_test[i] in t[i] for i in 1:length(labels_test)])

0.972005598880224

In [187]:
histogram(labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [188]:
histogram(predicted_labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1